In [1]:
import jax
import jax.numpy as np
import jraph
import flax.linen as nn
import numpy as onp
import matplotlib.pyplot as plt

from functools import partial

from jax.config import config
config.update("jax_debug_nans", True)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("../")

from models.gnn import GraphConvNet
from models.graph_utils import nearest_neighbors
from models.graph_utils import add_graphs_tuples
from models.train_utils import create_input_iter

In [114]:
from datasets import load_data

In [115]:
n_nodes = 10

In [116]:
train_ds, norm_dict = load_data("nbody", 3, n_nodes, 2, 234)

In [117]:
batches = create_input_iter(train_ds)
x = next(batches)
x = x[0][0]

In [118]:
k = 5

In [119]:
sources, targets = jax.vmap(nearest_neighbors, in_axes=(0, None))(x, k)

## EGNN

In [120]:
from models.egnn import EGNN

In [121]:
class GraphWrapper(nn.Module):
    @nn.compact
    def __call__(self, x):
        model = jax.vmap(EGNN(
                        message_passing_steps=2, d_hidden=64, n_layers=3, norm_layer=False, skip_connections=False,
                ))
        return model(x)
    
model = GraphWrapper()
rng = jax.random.PRNGKey(42)

In [126]:
np.array([np.arange(50)] * 2)

Array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
        32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
        48, 49],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
        32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
        48, 49]], dtype=int32)

In [ ]:
globals_ = np.array([np.arange(50)] * 2)

In [ ]:
tree.tree_map(lambda g: jnp.repeat(
        g, n_edge, axis=0, total_repeat_length=sum_n_edge), globals_)

In [125]:
graph = jraph.GraphsTuple(
          n_node=np.array(2 * [[n_nodes]]), 
          n_edge=np.array(2 * [[k]]),
          nodes=x, 
          edges=None,
          globals=np.array([np.arange(50)] * 2),
          # globals=np.ones((2,3)),
          senders=sources,
          receivers=targets)

graph_out, _ = model.init_with_output(rng, graph)

x_out = graph_out.nodes

eu (50, 3) (50, 3) (50, 1) Traced<ShapedArray(int32[50,1])>with<BatchTrace(level=1/0)> with
  val = Array([[[0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [2],
        [2],
        [2],
        [2],
        [2],
        [3],
        [3],
        [3],
        [3],
        [3],
        [4],
        [4],
        [4],
        [4],
        [4],
        [5],
        [5],
        [5],
        [5],
        [5],
        [6],
        [6],
        [6],
        [6],
        [6],
        [7],
        [7],
        [7],
        [7],
        [7],
        [8],
        [8],
        [8],
        [8],
        [8],
        [9],
        [9],
        [9],
        [9],
        [9]],

       [[0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [2],
        [2],
        [2],
        [2],
        [2],
        [3],
        [3],
        [3],
        [3],
 

In [127]:
x_out

Array([[[-0.68554604, -2.1479716 ,  1.5416772 ],
        [-1.2881864 ,  0.9780058 , -1.0715494 ],
        [-0.8396607 ,  0.75059104,  2.5186806 ],
        [-1.1226802 ,  0.77314615, -1.6128938 ],
        [-0.95244956, -1.0659342 , -0.4436468 ],
        [-1.2840563 , -0.8014892 , -1.1234771 ],
        [-1.2014519 , -1.4947162 ,  0.9071992 ],
        [-1.8943654 , -1.5812742 ,  1.3108653 ],
        [-1.2291371 , -1.3647547 ,  0.30546704],
        [-1.2536588 , -1.6653507 , -0.7905954 ]],

       [[-0.71604913, -0.33036906, -0.21970564],
        [-0.4039802 , -0.1996418 , -1.1914872 ],
        [-0.9214248 ,  0.13935837, -0.96085423],
        [-0.3744537 , -1.1275908 ,  1.023518  ],
        [-0.7268684 , -1.1289397 ,  1.1401561 ],
        [-0.3210816 , -1.3538364 ,  0.14696631],
        [ 0.02140472, -2.194496  , -1.5712606 ],
        [-0.11435717, -1.9374589 , -1.3566079 ],
        [-1.0350811 ,  1.0592834 , -0.24924001],
        [-1.075057  ,  1.6536064 , -0.77490246]]], dtype=float32)

## Actually run VDM

In [128]:
from models.diffusion import VariationalDiffusionModel
from flax.core import FrozenDict
from models.diffusion_utils import loss_vdm
from models.train_utils import create_input_iter, param_count, train_step

In [129]:
train_ds, norm_dict = load_data("nbody", 3, 5000, 4, 234)

In [22]:
batches = create_input_iter(train_ds)

In [23]:
x, conditioning, mask = next(batches)

In [24]:
score_dict = FrozenDict({"k":20, "message_passing_steps":2, "skip_connections":False, "norm_layer":False, "n_layers":4, "d_hidden":16, "n_pos_features":3})  # GNN args
encoder_dict = decoder_dict = FrozenDict({})
score = "egnn"

vdm = VariationalDiffusionModel(
          timesteps=0, 
          d_t_embedding=16,
          d_feature=3,
          score=score,
          score_dict=score_dict,
          n_classes=2,
          embed_context=True,
          d_context_embedding=16,
          noise_schedule="learned_linear",
          gamma_min=-8.,
          gamma_max=14.,
          use_encdec=False,
          encoder_dict=encoder_dict,
          decoder_dict=decoder_dict)

In [25]:
x.shape, conditioning.shape, mask.shape

((1, 4, 5000, 3), (1, 4, 2), (1, 4, 5000))

In [26]:
# Pass a test batch through to initialize model
n_smoke = 4

x = np.array(x[0, :n_smoke])
conditioning = np.array(conditioning[0, :n_smoke])
mask = np.array(mask[0, :n_smoke])

rng = jax.random.PRNGKey(42)
out, params = vdm.init_with_output({"sample": rng, "params": rng}, x, conditioning, mask);

In [27]:
loss = loss_vdm(params, vdm, rng, x, np.zeros_like(conditioning), mask)

In [28]:
loss, loss_grad = jax.value_and_grad(loss_vdm)(params, vdm, rng, x, np.zeros_like(conditioning), mask)

## EGNN other implemen